### Libraries, paths, and set-up

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia')
import src.utils.utils
import src.utils.calculations
from src.data.loader import data_loader
import warnings
warnings.filterwarnings('ignore')

storing_path = 'data/processed/'

In [9]:
soi, capacity, resource = data_loader('soi','capacity', 'resource')
market = pd.read_csv('data/processed/market.csv')

### Increasing SOI granularity to hourly

In [10]:
# Convert 'date' column to datetime format
soi_h = soi.copy()
soi_h = datetimer(soi_h, col_name='date')

# set 'date' as index
soi_h.set_index('date', inplace=True)

# resample data to an hourly frequency
soi_h = soi_h.resample('H').asfreq()
soi_h = soi_h.fillna(method='ffill')
soi_h = soi_h.reset_index(drop=False)
soi_h = soi_h.rename(columns={'date':'datetime'})
soi_h.to_csv(storing_path + 'h_soi.csv')

### Merging SOI information to market information

In [11]:
posmarket = market[market['markup'] > 0]
iqrmarket = iqr_filter_yearly(posmarket, 'markup')

In [12]:
iqrmarket = datetimer(iqrmarket)
iqrmarketsoi = iqrmarket.merge(soi_h, how='left', on='datetime')

# Replacing these plants since they were grouped after 2011
iqrmarketsoi['plant'] = iqrmarketsoi['plant'].replace({'FLORES 2':'FLORES 4 CC',
                                                      'FLORES 3':'FLORES 4 CC'})

fulldata = iqrmarketsoi.merge(resource[['plant','heat_rate','agent','technology','fuel','netcapacity_kW']], how='left', on='plant')

fulldata = fulldata.drop(columns=['year','month','agent_code'])

fulldata.to_csv(storing_path + 'fulldata.csv', index=False)

In [13]:
fulldata.isna().sum()

plant                 0
daily_ask             0
datetime              0
supply_hourly         0
demand_hourly         0
hourly_bid            0
hourly_mc             0
total_supply_agent    0
total_supply_t        0
residual_supply       0
rsi_agent             0
market_share_agent    0
markup                0
lerner                0
comp_markup           0
comp_lerner           0
soi                   0
heat_rate             0
agent                 0
technology            0
fuel                  0
netcapacity_kW        0
dtype: int64